# Imports

In [32]:
!pip install -r requirements.txt

In [ ]:
#TODO: When everything done, upload to git so that can be cloned

In [33]:
from argparse import ArgumentParser
import random

import dill as pickle

import sys
import yaml
import torchmetrics
from typing import Tuple
from pytorch_lightning.strategies import DeepSpeedStrategy
import warnings
import os
import uuid
import models
import utils
from utils import forecasting_metrics
from utils import plot
import pandas as pd
import data
import pytorch_lightning as pl
import torch
import random
from typing import List
import os
import tqdm
import datetime
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

from torch import nn
import numpy as np
import torch.nn.functional as F
from datetime import datetime
import wandb
import json
from pytorch_lightning.callbacks import TQDMProgressBar
from abc import ABC, abstractmethod

In [34]:
with open('config.yaml', 'r') as stream:
	config=yaml.safe_load(stream)
run_name = f"STT{config['steps_to_predict']}h-PM2.5-{datetime.now().strftime('%d.%m.%Y-%H:%M:%S')}"

## Forecaster

In [35]:
forecaster = models.Forecaster(config)

# Creating the dataset

In [36]:
df=pd.read_csv("data/helsinkipm25.csv",delimiter=";", decimal=",")
df.iloc[:,:-1] = df.iloc[:,:-1].apply(pd.to_numeric,errors='coerce') # everything to numeric

In [37]:
df.head()

,luukki,kallio,mannerheimintie,makelankatu,leppavaara,lohja,tikkurila,vartiokyla,Datetime
0,5.5,38.7,56.8,NaN,34.6,4.8,13.3,41.5,2015-01-01 00:00:00
1,9.8,13.2,25.8,NaN,16.1,3.8,6.6,26.8,2015-01-01 01:00:00
2,9.1,10.2,18.9,NaN,13.8,3.7,6.6,22.4,2015-01-01 02:00:00
3,9.7,7.4,13.9,NaN,9.7,3.6,7.4,18.5,2015-01-01 03:00:00
4,3.8,6.6,13.0,NaN,7.6,3.4,6.9,15.8,2015-01-01 04:00:00


In [38]:
# Replace negative values in other columns 0-7 with 0
df.iloc[:,1:8] = df.iloc[:,1:8].apply(lambda x: x.clip(lower=0))

In [39]:
inv_scaler = lambda x: x
scaler = lambda x: x
null_val = None
plot_var_idxs = None
plot_var_names = None
pad_val = None
target_cols = df.columns.values.tolist()[:config["d_yc"]]
dset = data.CSVTimeSeries(
    raw_df=df,
    target_cols=target_cols,
    time_col_name="Datetime"
)

In [40]:
data_module = data.DataModule(
    datasetCls=data.CSVTorchDset,
    dataset_kwargs={
        "csv_time_series": dset,
        "context_points": config["context_points"],
        "target_points": config["target_points"],
        "time_resolution": config["time_resolution"],
    },
    batch_size=config["batch_size"],
    workers=config["workers"],
    overfit=config["overfit"],
)
inv_scaler = dset.reverse_scaling
scaler_ = dset.apply_scaling
null_val = None

In [41]:
dset._train_data.head(5)

,luukki,kallio,mannerheimintie,makelankatu,leppavaara,lohja,tikkurila,vartiokyla,Datetime,Year,Month,Day,Weekday,Hour,Minute
0,0.212165,7.931994,9.877799,NaN,6.953384,0.026170,1.358058,7.496525,2015-01-01 00:00:00,0.0,-1.0,-1.0,0.0,-1.000000,-1.0
1,1.284990,1.832112,3.777126,NaN,2.518413,-0.225569,0.065572,4.404372,2015-01-01 01:00:00,0.0,-1.0,-1.0,0.0,-0.913043,-1.0
2,1.110344,1.114478,2.419235,NaN,1.967038,-0.250743,0.065572,3.478830,2015-01-01 02:00:00,0.0,-1.0,-1.0,0.0,-0.826087,-1.0
3,1.260040,0.444687,1.435255,NaN,0.984153,-0.275917,0.219898,2.658463,2015-01-01 03:00:00,0.0,-1.0,-1.0,0.0,-0.739130,-1.0
4,-0.211975,0.253318,1.258139,NaN,0.480724,-0.326265,0.123444,2.090517,2015-01-01 04:00:00,0.0,-1.0,-1.0,0.0,-0.652174,-1.0


## Model run

In [42]:
forecaster.set_inv_scaler(inv_scaler)
forecaster.set_scaler(scaler)
forecaster.set_null_value(null_val)

test_samples = next(iter(data_module.test_dataloader()))

if config["val_check_interval"] <= 1.0:
    val_control = {"val_check_interval": config["val_check_interval"]}
else:
    val_control = {"check_val_every_n_epoch": int(config["val_check_interval"])}

# Save the variables to a file to run the model in a non-interactive environment (outside of the notebook)
with open("trainer_vars.pkl", "wb") as f:
    pickle.dump({"forecaster": forecaster,"test_samples": test_samples,"data_module": data_module, "run_name": run_name, "config": config, "val_control": val_control}, f)

In [45]:
!python trainer.py

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: danielofosu. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.9
wandb: Run data is saved locally in ./data/wandb_logs/wandb/run-20230124_150649-g52gff1a
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run sweet-moon-533
wandb: ⭐️ View project at https://wandb.ai/danielofosu/thesis
wandb: 🚀 View run at https://wandb.ai/danielofosu/thesis/runs/g52gff1a
wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
initializing deepspeed distributed: GLOBAL_RANK: 0, MEMBER: 1/1
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Installed CUDA version 11.2 

In [44]:
#unpickle first
# Predictions

forecaster.to("cuda")
xc, yc, xt, _ = test_samples
yt_pred = forecaster.predict(xc, yc, xt)

#yt_pred 3d tensor to dataframe
yt_pred = yt_pred.cpu().detach().numpy()
#yt_pred[1] array to dataframe
yt_pred = pd.DataFrame(yt_pred[1])
#yt_pred cols from original cols
yt_pred.columns = target_cols

#array from xc yc xt
timeseriesdata = xt.cpu().detach().numpy()
timeseriesdata = pd.DataFrame(timeseriesdata[1])

AttributeError: 'Forecaster' object has no attribute 'predict'

In [ ]:


# set timeseries data column names to Year	Month	Day	Weekday	Hour	Minute
timeseriesdata.columns = ['Day', 'Hour', 'Minute', 'Month', 'Weekday', 'Year']
testdf= dset.test_data
timeseriesdata.head(5)
#round timeseriesdata day to four decimals
timeseriesdata = timeseriesdata.round(4)
testdf = testdf.round(4)
timeseriesdata

In [ ]:

testdf[testdf['Datetime'] == "2020-12-21 08:00:00"]
yt_real = testdf.iloc[8952:8952+80]
yt_real = yt_real.iloc[:,0:8]
#yt real inverse scaling
yt_real = inv_scaler(yt_real)
yt_real.reset_index(drop=True, inplace=True)

#Adding datetimes
yt_pred["Datetime"] = testdf.iloc[8952:8952+80]["Datetime"].values
yt_real["Datetime"] = testdf.iloc[8952:8952+80]["Datetime"].values
yt_real.head(5)

In [ ]:
yt_pred.head(5)